 ## Auto Encoders

### The Plan
1. Split data into gene and cell features
2. Train a AE on each
3. Use AE to generate latent features for gene and cell features
4. Use to train ensemble (NN, LR, XGB)

In [1]:
import os
import random
import time
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os.path import join, exists
from torch import nn
from torch import optim
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

import sys
sys.path.append('..')
from pipeline.data import MoaDataset
from pipeline.networks import MoaDenseNet, DenseModule, initialize_weights, recalibrate_layer
from pipeline.utils import SETTINGS

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [3]:
SEED = 123
BATCH_SIZE = 1024
N_FOLD = 5

seed_everything(SEED)

In [4]:
train_features = pd.read_csv('../data/train_features.csv')
train_targets = pd.read_csv('../data/train_targets_scored.csv')
train_targets_ns = pd.read_csv('../data/train_targets_nonscored.csv')
test_features = pd.read_csv('../data/test_features.csv')

In [5]:
features = [c for c in train_features.columns if '-' in c]
all_features = np.vstack([train_features[features].values, test_features[features].values])
scaler = StandardScaler()
features_t = scaler.fit_transform(all_features)
scaler_2 = MinMaxScaler()
features_t = scaler_2.fit_transform(features_t)

In [6]:
def process_df(df):
    df.loc[:, 'cp_time'] = df.loc[:, 'cp_time'].map({24: 0, 48: 0.5, 72: 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    df = df.drop('cp_type', axis=1)
    df.loc[:, features] = scaler_2.transform(scaler.transform(df.loc[:, features]))
    return df

In [7]:
train_features = process_df(train_features)
test_features = process_df(test_features)

In [8]:
class Metrics:
    def __init__(self):
        self.data = []

    def load(self, data):
        self.data = data

    def add(self, x):
        return self.data.append(x)

    def mean(self):
        return np.mean(self.data)

    def min_epoch(self):
        return np.argmin(self.data)

    def min(self):
        return np.min(self.data)

    def max(self):
        return np.max(self.data)

    def tail(self):
        return self.data[-1]

    def reset(self):
        self.data = []

In [9]:
class AutoEncoderDataset(Dataset):
    def __init__(self, x):
        self.x = x
        self.length = len(self.x)
    
    def __getitem__(self, idx):
        return self.x[idx], self.x[idx]
    
    def __len__(self):
        return self.length

In [10]:
class AutoEncoder(nn.Module):
    def __init__(self, feature_size, latent_layer_size, init_dropout, dropout, normalization, activation):
        super(AutoEncoder, self).__init__()
        
        bounds = (feature_size, latent_layer_size)
        intermediate_layer_size = int(((max(bounds) - min(bounds)) / 2) + min(bounds))
        
        self.dense_0 = DenseModule(feature_size, intermediate_layer_size, init_dropout, normalization, activation)
        self.dense_1 = DenseModule(intermediate_layer_size, latent_layer_size, dropout, normalization, activation)
        self.dense_2 = DenseModule(latent_layer_size, intermediate_layer_size, dropout, normalization, activation)
        self.dense_3 = DenseModule(intermediate_layer_size, feature_size, dropout, normalization, activation=None)
        
    def forward(self, x):
        latent = self.dense_1(self.dense_0(x))
        logits = self.dense_3(self.dense_2(latent))
        return latent, logits

In [25]:
class NeuralNetModel:
    def __init__(self, model_dict):
        self.model_dict = model_dict
        self.best_weights_path = join(self.model_dict['model_dir'], 'fold-%d-weights.pth' % self.model_dict['fold'])
        self.epoch = 0
        self.criterion = nn.BCEWithLogitsLoss()
        self.train_metrics = Metrics()
        self.valid_metrics = Metrics()

        if model_dict['normalization'] == 'batch':
            model_dict['normalization'] = nn.BatchNorm1d
        else:
            Exception('Normalization not supported.')

        if model_dict['activation'] == 'relu':
            model_dict['activation'] = nn.ReLU
        elif model_dict['activation'] == 'prelu':
            model_dict['activation'] = nn.PReLU
        else:
            Exception('Activation not supported.')
        
        if model_dict['model'] == 'AutoEncoder':
            self.model = AutoEncoder(
                model_dict['feature_size'],
                model_dict['latent_layer_size'],
                model_dict['init_dropout'],
                model_dict['dropout'],
                model_dict['normalization'],
                model_dict['activation'],
            )
        elif model_dict['model'] == 'MoaDenseNet':
            self.model = MoaDenseNet(
                model_dict['input_dim'],
                model_dict['output_dim'],
                model_dict['n_hidden_layer'],
                model_dict['hidden_dim'],
                model_dict['dropout'],
                model_dict['activation'],
                model_dict['normalization'],
            )

        # Setup optimizer
        if model_dict['optimizer'] == 'sgd':
            self.optimizer = optim.SGD(self.model.parameters(),
                                       lr=model_dict['learning_rate'],
                                       momentum=model_dict['momentum'])
        elif model_dict['optimizer'] == 'adam':
            self.optimizer = optim.Adam(self.model.parameters(),
                                        lr=model_dict['learning_rate'],
                                        weight_decay=model_dict['weight_decay'])
        else:
            Exception('Optimizer not supported.')

        # Setup scheduler
        if model_dict['scheduler'] == 'ReduceLROnPlateau':
            self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(self.optimizer,
                                                                  patience=3,
                                                                  threshold=0.00001)
        elif model_dict['scheduler'] == 'OneCycleLR':
            self.scheduler = optim.lr_scheduler.OneCycleLR(self.optimizer,
                                                           max_lr=model_dict['max_lr'],
                                                           pct_start=model_dict['pct_start'],
                                                           div_factor=model_dict['div_factor'],
                                                           epochs=model_dict['n_epochs'],
                                                           steps_per_epoch=model_config['steps_per_epoch'])
        else:
            Exception('Scheduler not supported.')

        # Save initial states of model, optimizer and scheduler
        self.init_states = dict(
            model=self.model.state_dict(),
            optimizer=self.optimizer.state_dict(),
            scheduler=self.scheduler.state_dict()
        )
        self.model = self.model.cuda()

    def unwrap_batch(self, batch):
        if self.model_dict['model'] == 'AutoEncoder':
            features_c, features = batch
            features_c = features_c.cuda().float()
            features = features.cuda().float()
        elif self.model_dict['model'] == 'MoaDenseNet':
            features, targets, ids = batch
            features = {k: v.cuda().float() for k, v in features.items()}
            targets = targets.cuda().float()
        return features, targets
        
    def train_epoch(self, train_dataloader):
        losses = []
        self.model.train()

        for batch in train_dataloader:
            features, targets = self.unwrap_batch(batch)
            
            output = self.model(features)
            
            loss = self.criterion(output['prediction'], targets)
            for p in self.model.parameters():
                p.grad = None
            loss.backward()
            self.optimizer.step()
            if self.model_dict['scheduler'] == 'OneCycleLR':
                self.scheduler.step()
            
            losses.append(loss.detach().cpu().numpy())
        
        avg_loss = float(np.mean(losses))
        self.train_metrics.add(avg_loss)
        return avg_loss

    def validation(self, valid_dataloader, return_preds=False):
        losses = []
        predictions = []
        self.model.eval()

        for batch in valid_dataloader:
            features, targets = self.unwrap_batch(batch)
            
            output = self.model(features)
            loss = self.criterion(output['prediction'], targets)
            losses.append(loss.detach().cpu().numpy())
            if return_preds:
                predictions.extend(output['prediction'].detach().cpu().numpy())

        avg_loss = float(np.mean(losses))

        if return_preds:
            return avg_loss, predictions
        else:
            return avg_loss

    def train(self, train_dataloader, valid_dataloader):
        for epoch in range(self.model_dict['n_epochs']):
            self.epoch = epoch
            time0 = time.time()

            train_avg_loss = self.train_epoch(train_dataloader)
            valid_avg_loss = self.validation(valid_dataloader)

            if self.epoch == 0 or valid_avg_loss < self.valid_metrics.min():
                # new best weights
                self.save(self.best_weights_path)
            self.valid_metrics.add(valid_avg_loss)

            if self.model_dict['scheduler'] != 'OneCycleLR':
                self.scheduler.step(valid_avg_loss)

            time1 = time.time()
            epoch_time = time1 - time0
            if self.model_dict['verbose']:
                print('Epoch %d/%d Train Loss: %.5f Valid Loss: %.5f Time: %.2f' % (
                    epoch+1, self.model_dict['n_epochs'], train_avg_loss, valid_avg_loss, epoch_time
                ))

        # restore weights from best epoch
        self.load(self.best_weights_path)

    def predict(self, test_features):
        test_dataset = MoaDataset(test_features)
        test_dataloader = DataLoader(test_dataset,
                                     batch_size=self.model_dict['batch_size'],
                                     num_workers=self.model_dict['num_workers'],
                                     pin_memory=True)
        predictions = []
        self.model.eval()
        for features, ids in test_dataloader:
            features = {k: v.cuda().float() for k, v in features.items()}
            pred = self.model(features)
            predictions.extend(pred.detach().cpu().numpy())
        return predictions

    def reset(self):
        self.model.load_state_dict(self.init_states['model'])
        self.optimizer.load_state_dict(self.init_states['optimizer'])
        self.scheduler.load_state_dict(self.init_states['scheduler'])
        self.train_metrics.reset()
        self.valid_metrics.reset()

    def save(self, path):
        self.model.eval()
        torch.save({
            'model': self.model.state_dict(),
            'optimizer': self.optimizer.state_dict(),
            'scheduler': self.scheduler.state_dict(),
            'train_metrics': self.train_metrics.data,
            'valid_metrics': self.valid_metrics.data,
        }, path)

    def load(self, path):
        state_dict = torch.load(path)
        self.model.load_state_dict(state_dict['model'])
        self.optimizer.load_state_dict(state_dict['optimizer'])
        self.scheduler.load_state_dict(state_dict['scheduler'])
        self.train_metrics.load(state_dict['train_metrics'])
        self.valid_metrics.load(state_dict['valid_metrics'])

In [12]:
model_config = dict(
    model='AutoEncoder',
    latent_layer_size=64,
    init_dropout=0.2,
    dropout=0.25,
    normalization='batch',
    activation='prelu',
    
    n_epochs=150,
    optimizer='adam',
    learning_rate=0.001,
    weight_decay=1e-5,
    scheduler='OneCycleLR',
    verbose=True,
    max_lr=0.05,
    pct_start=0.05,
    div_factor=1e3
)

In [13]:
# # Train each autoencoder
# for label in ['gene', 'cell']:
        
#     # Setup model directory
#     model_dir = join(SETTINGS['PROJECTS_DIR'], '%s_autoencoder' % label)
#     if not exists(model_dir):
#         os.mkdir(model_dir)
    
#     # Load data
#     features = ['cp_time', 'cp_dose'] + [c for c in train_features.columns if label[0] + '-' in c]
#     x = train_features[features].values
#     test_x = test_features[features].values
    
#     # Setup dataloaders
#     train_dataset = AutoEncoderDataset(x)
#     train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, pin_memory=True, num_workers=4, shuffle=True)
#     test_dataset = AutoEncoderDataset(test_x)
#     test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, pin_memory=True, num_workers=4)

#     # Pass variables through model configuration
#     model_config['feature_size'] = x.shape[1]
#     model_config['model_dir'] = model_dir
#     model_config['fold'] = 0
#     model_config['steps_per_epoch'] = len(train_dataloader)

#     # Initialize model
#     model = NeuralNetModel(model_config)

#     # Set custom init weights for fast convergence
#     init_bias = np.mean(x, axis=0)
#     weights = model.model.dense_3.state_dict()
#     weights['dense.bias'] = torch.tensor(init_bias)
#     model.model.dense_3.load_state_dict(weights)

#     # Train model
#     model.train(train_dataloader, test_dataloader)

#     # Gather losses
#     loss = model.valid_metrics.min()
#     print('%.5f' % loss)

In [14]:
# label = 'gene'
# model_dir = join(SETTINGS['PROJECTS_DIR'], '%s_autoencoder' % label)

# features = ['cp_time', 'cp_dose'] + [c for c in train_features.columns if label[0] + '-' in c]
# test_x = test_features[features].values
# test_dataset = AutoEncoderDataset(test_x)
# test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, pin_memory=True, num_workers=4)

# # Pass variables through model configuration
# model_config['feature_size'] = test_x.shape[1]
# model_config['model_dir'] = model_dir
# model_config['fold'] = 0
# model_config['steps_per_epoch'] = len(test_dataset)

# # Initialize model
# model = NeuralNetModel(model_config)
# model.load(model.best_weights_path)
# loss, preds = model.validation(test_dataloader, return_preds=True)

In [15]:
# crit = nn.BCEWithLogitsLoss()

# fig, ax = plt.subplots(5, 5, figsize=(20, 20))
# for i in range(25):
#     rand_idx = int(np.random.random()*len(test_dataset))
#     ground_truth = torch.tensor(test_dataset[rand_idx][0])
#     prediction = torch.tensor(preds[rand_idx])
#     loss = crit(prediction, ground_truth)
#     prediction = nn.Sigmoid()(prediction).numpy()
    
#     ax[i//5, i%5].scatter(ground_truth, prediction)
#     ax[i//5, i%5].set_xlim(0,1)
#     ax[i//5, i%5].set_ylim(0,1)
#     ax[i//5, i%5].set_title('loss: %.3f' % (loss))
    
# plt.show()

In [17]:
# Get latent feature vectors for train and test set

In [18]:
# sig = nn.Sigmoid()

# data = {}
# for label in ['gene', 'cell']:
#     model_dir = join(SETTINGS['PROJECTS_DIR'], '%s_autoencoder' % label)
    
#     # Load data
#     features = ['cp_time', 'cp_dose'] + [c for c in train_features.columns if label[0] + '-' in c]
#     x = train_features[features].values
#     test_x = test_features[features].values

#     # Setup dataloaders
#     train_dataset = AutoEncoderDataset(x)
#     train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, pin_memory=True, num_workers=4)
#     test_dataset = AutoEncoderDataset(test_x)
#     test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, pin_memory=True, num_workers=4)

#     # Pass variables through model configuration
#     model_config['feature_size'] = test_x.shape[1]
#     model_config['model_dir'] = model_dir
#     model_config['fold'] = 0
#     model_config['steps_per_epoch'] = len(test_dataset)

#     # Initialize model
#     model = NeuralNetModel(model_config)
#     model.load(model.best_weights_path)
#     loss, preds = model.validation(train_dataloader, return_preds=True)
#     data[label+'_train'] = sig(torch.tensor(preds)).numpy()
    
#     loss, preds = model.validation(test_dataloader, return_preds=True)
#     data[label+'_test'] = sig(torch.tensor(preds)).numpy()
    
# train_data = np.hstack([np.expand_dims(train_features['sig_id'].values, 1), data['gene_train'], data['cell_train']])
# test_data = np.hstack([np.expand_dims(test_features['sig_id'].values, 1), data['gene_test'], data['cell_test']])
# columns = ['sig_id'] + ['f-%d' % i for i in range(train_data.shape[1] - 1)]
# train_ae_features = pd.DataFrame(train_data, columns=columns)
# test_ae_features = pd.DataFrame(test_data, columns=columns)
# train_ae_features.to_csv('train_ae_features.csv', index=False)
# test_ae_features.to_csv('test_ae_features.csv', index=False)

In [19]:
train_ae_features = pd.read_csv('train_ae_features.csv')
test_ae_features = pd.read_csv('test_ae_features.csv')
y = train_targets.values[:, 1:]
skf = MultilabelStratifiedKFold(n_splits=N_FOLD, shuffle=True, random_state=SEED)
test_dataset = MoaDataset(test_ae_features)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, pin_memory=True, num_workers=4)

/home/matthew/anaconda3/envs/ml_py385/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass shuffle=True, random_state=123 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [20]:
model_config = dict(
    model='MoaDenseNet',
    n_hidden_layer=1,
    dropout=0.5,
    hidden_dim=1024,    
    normalization='batch',
    activation='relu',
    
    n_epochs=25,
    optimizer='adam',
    learning_rate=0.001,
    weight_decay=1e-5,
    scheduler='OneCycleLR',
    verbose=True,
    max_lr=0.05,
    pct_start=0.05,
    div_factor=1e3
)

In [27]:
model_dir = join(SETTINGS['PROJECTS_DIR'], 'ae_classifier')
if not exists(model_dir): os.mkdir(model_dir)

In [28]:
for fold_i, (train_idx, valid_idx) in enumerate(skf.split(train_ae_features, y)):
    train_x, train_y = train_ae_features.iloc[train_idx], train_targets.iloc[train_idx]
    valid_x, valid_y = train_ae_features.iloc[valid_idx], train_targets.iloc[valid_idx]
    
    # Setup dataloaders
    train_dataset = MoaDataset(train_x, train_y)
    train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, pin_memory=True, num_workers=4, shuffle=True)
    valid_dataset = MoaDataset(valid_x, valid_y)
    valid_dataloader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, pin_memory=True, num_workers=4)

    # Pass variables through model configuration
    model_config['input_dim'] = len(train_x.columns) - 1
    model_config['output_dim'] = len(train_y.columns) - 1
    model_config['model_dir'] = model_dir
    model_config['fold'] = fold_i
    model_config['steps_per_epoch'] = len(train_dataset)
    
    # train neural network
    model = NeuralNetModel(model_config)
    model.model.layers = initialize_weights(model.model.layers)
    model.train(train_dataloader, valid_dataloader)
    
    
    # logistic regression
    
    # xgboost

Epoch 1/25 Train Loss: 0.02186 Valid Loss: 0.02084 Time: 0.94
Epoch 2/25 Train Loss: 0.02074 Valid Loss: 0.02091 Time: 0.79
Epoch 3/25 Train Loss: 0.02038 Valid Loss: 0.02064 Time: 0.83
Epoch 4/25 Train Loss: 0.02010 Valid Loss: 0.02006 Time: 0.89
Epoch 5/25 Train Loss: 0.01995 Valid Loss: 0.01947 Time: 0.83
Epoch 6/25 Train Loss: 0.01982 Valid Loss: 0.01914 Time: 0.87
Epoch 7/25 Train Loss: 0.01966 Valid Loss: 0.01903 Time: 0.88
Epoch 8/25 Train Loss: 0.01961 Valid Loss: 0.01896 Time: 0.84
Epoch 9/25 Train Loss: 0.01948 Valid Loss: 0.01892 Time: 0.86
Epoch 10/25 Train Loss: 0.01941 Valid Loss: 0.01888 Time: 0.83
Epoch 11/25 Train Loss: 0.01938 Valid Loss: 0.01885 Time: 0.82
Epoch 12/25 Train Loss: 0.01932 Valid Loss: 0.01882 Time: 0.86
Epoch 13/25 Train Loss: 0.01928 Valid Loss: 0.01879 Time: 0.82
Epoch 14/25 Train Loss: 0.01919 Valid Loss: 0.01876 Time: 0.86
Epoch 15/25 Train Loss: 0.01917 Valid Loss: 0.01874 Time: 0.84
Epoch 16/25 Train Loss: 0.01911 Valid Loss: 0.01870 Time: 0.80
E